# Generating summary statistics for `Porsche Auction Data.xlsx`

Run each of these cells in order by pressing the `Run` button above or by click on a cell and typing `CTRL`+`ENTER`

This first cell has notebook configuration options that make visualizations prettier and make it easier for me to develop code in separate script files to use in this notebook. Always run this cell first.

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Import libraries

These cells import Python libraries. 

`pandas` is a library for efficiently managing and querying 2-D and 1-D 'spreadsheet'-like data.

In [ ]:
import pandas as pd

`numpy` is a Python library for general numerical analysis/computation. It also contains some basic data manipulation and statistical functions which we'll use here

In [ ]:
import numpy as np

`plotly` is a library for data visualization in Python. For now we will use the parts of the ecosystem for creating simple scatter plots, box plots, etc. However we can also experiment with [Dash](https://plot.ly/dash/) to create web-based interactive dashboards for working with the data in the future.

In [ ]:
import plotly.graph_objects as go

The `import <lib> as <name>` structure is designed so that people can rename libraries they are importing in case they already have things in their code with the same name. In practice, it is often used for shortening the names of libraries so code is more compact. Renaming `pandas` to "pd" and `numpy` to "np" is a pretty common practice/tradition in the Python community.

# Load the data

This cell loads our spreadsheet. It uses a function in the `pandas` library `read_excel()` to import data stored in an Excel file and represent it in a data structure (called a `DataFrame`) that can be interacted with using Python code. 

This object is referenced using the variable `data` that we are defining below. Typically we would want to use a more descriptive variable name, but we are only working with one spreadsheet in this notebook, and we will see that giving `DataFrame` objects relatively short names is a plus once we start working with them.

A note on the file path below - inside a Jupyter server there is always one user by default named "jovyan", an anonymous [resident of the planet Jupyter](https://en.wikipedia.org/wiki/Jovian_(fiction)). By default all of your data lives in `/home/jovyan/`.

In [ ]:
data = pd.read_excel("/home/jovyan/test_data/Porsche Auction Data.xlsx")

# A quick preview

It's a good idea to check that our data was loaded in and represented how we expect it to be. For this we can use a pandas function called `head()` that prints the first five rows by default.

In [ ]:
data.head()

Looks good! `pandas` found the column names without any problem, and because we didn't tell it what to use as an *index*, it just gave each row an integer ID (bold numbers on the lefthand side). Normally we would do something like `data = pd.read_excel("data.xlsx", index="ID/VIN")`, but since our data is incomplete, this isn't a good idea. Addressing this issue will be one part of cleaning the data, especially if we want to query this data with respect to other datasets that might have the same vehicle IDs.

# What auction house sells the most expensive cars?

I will use the next few code cells build a boxplot to answer this question.

## Get a list of all the auction houses in the data

To filter the data by auction house, we need to know all the names. This line of code retrieves the `House` column and uses the `set()` function to remove repeated names and make a collection of each of the individual auction houses in the data. It then converts them to a `list` mostly because I like `list` objects and find `set` objects to be kind of annoying in most cases.

In [ ]:
houses = list(set(data["House"]))
print(houses)

Look carefully and you can see another issue that we'll have to preprocess out at some point, an unfortunate addition of space characters has separated "Russo and Steele" into two different houses.

We can use a function called `len()` to check how many elements are in our list.

In [ ]:
print("There are {} auction houses in our dataset... well actually {}".format(len(houses), len(houses)-1))

## Getting prices from each house

Now that we have our `list` of auction houses, the most Python-ish thing to do next is *iterate* over them to perform some calculation. We'll use a programming contruct called a `for` loop common to many languages that essentially specifies a set of computations to perform on each object in a collection. Here's a simple example.

In [ ]:
for house in houses:
    print("I am an auction house and my name is {}".format(house))

We'll use the same *control flow* to assemble all the pricing data for individual auction houses. First we will define a dictionary, or `dict`, another type of Python data structure designed for efficient storage of *key/value* pairs, to store lists of prices, each list keyed by an auction house name.

In [ ]:
house_prices = {}  ## this will look something like {"RM": [45000, 23000, 34500, ...], ...}

for house in houses:
    house_prices[house] = list(data[data["House"]==house]["Price"])

In [ ]:
for house in houses:
    print("The first five prices for house {} are {}".format(house, house_prices[house][:5]))

We can see that there are some entries in the `Price` column that are definitely not numbers. This is another issue with the data that we will have to be remedied. For now, we can filter our `DataFrame` to only contain rows that have a numerical price.

In [ ]:
print("Number of rows before filtering: {}".format(data.shape[0]))

In [ ]:
data_numeric = data[data["Price"].apply(lambda x: type(x) in [int, np.int64])]

In [ ]:
print("Number of rows after filtering: {}".format(data_numeric.shape[0]))

We lost less than 100 rows, so this isn't the end of the world for now, we can continue. Let's run the same code again to produce our dictionary of houses and their associated prices. The only difference is that we will use `data_numeric` instead of `data`.

In [ ]:
house_prices = {}  ## this will look something like {"RM": [45000, 23000, 34500, ...], ...}

for house in houses:
    house_prices[house] = list(data_numeric[data_numeric["House"]==house]["Price"])

In [ ]:
for house in houses:
    print("The first five prices for house {} are {}".format(house, house_prices[house][:5]))

## Visualization

We've done 90% of the hard work - wrangling the data into a useful form. This code will use it to create a boxplot visualization of prices associated with each house.

In [ ]:
house_price_fig = go.Figure()

for house in houses:
    house_price_fig.add_trace(
        go.Box(
            y=house_prices[house],
            name=house
        )
    )
    
house_price_fig.show()

Looks like Gooding brings in the high spenders! This is an interactive plot, try mousing over the boxes and outlier points. Also, try grabbing a section of the plot as if you were cropping it to zoom in (zoom out by double clicking anywhere on the plot). This is a helpful feature since our plot here is dominated by Gooding and their multi-million dollar cars. You can also save the plot by mousing over the top right corner and clicking on the camera icon.

# Future work

Whatever you want next! I can start by installing [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) in the Docker container and writing a parser for the auction sites.